Importing Libraries

In [2]:
import pandas as pd
import numpy as np
import re

Importing data and printing first 10 rows

In [3]:
df = pd.read_csv('/content/train.csv')
df.head(5)

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
1,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13 km/kg,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
2,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08 kmpl,1461 CC,63.1 bhp,5.0,NaN,3.50


a) Look for the missing values in all the columns and either impute them (replace with mean,
median, or mode) or drop them. Justify your action for this task.

In [4]:
# Total missing values
missing_count = df.isnull().sum()

# Percentage of missing values
missing_percentage = (missing_count / len(df)) * 100

# Combine both into a single DataFrame
missing_info = pd.DataFrame({
    'Missing Values': missing_count,
    'Percentage (%)': missing_percentage.round(2)
})

# Show only rows where there's at least one missing value
missing_info = missing_info[missing_info['Missing Values'] > 0]

print(missing_info)


           Missing Values  Percentage (%)
Mileage                 2            0.03
Engine                 36            0.62
Power                  36            0.62
Seats                  38            0.65
New_Price            5032           86.06


Replacing the mising values

In [5]:
# Making a copy after dropping missing mileage rows to avoid SettingWithCopyWarning
df = df[df['Mileage'].notnull()].copy()

# Cleaning 'Engine' and fill with mode
df['Engine'] = df['Engine'].astype(str).str.replace(' CC', '', regex=False)
df['Engine'] = pd.to_numeric(df['Engine'], errors='coerce')
df.loc[:, 'Engine'] = df['Engine'].fillna(df['Engine'].mode()[0])

# Cleaning 'Power' and converting to numeric
df['Power'] = df['Power'].astype(str).str.replace(' bhp', '', regex=False)
df['Power'] = pd.to_numeric(df['Power'], errors='coerce')
df.loc[:, 'Power'] = df['Power'].fillna(df['Power'].mean())

# Filling missing 'Seats' with mode
df.loc[:, 'Seats'] = df['Seats'].fillna(df['Seats'].mode()[0])

# Droping 'New_Price' column
df.drop(columns=['New_Price'], inplace=True)


**Justification**

**Dropping Mileage rows:** The number of missing values is relatively small and Mileage is considered an important feature, it's reasonable to drop those rows to avoid introducing bias through imputation.

**Imputing Engine with Mode:** Engine sizes are often categorical (e.g., 800cc, 1000cc), so mode makes more sense

**Imputing Power with Mean:** Power is a continuous numerical variable (measured in bhp, or brake horsepower), making mean imputation statistically appropriate.

**Imputing Seatings with mode:** 	Number of seats is categorical

In [6]:
# Check missing values in each column
missing_values = df.isnull().sum()

# Show only columns with missing values
missing_values = missing_values[missing_values >= 0]

print("Missing values in each column:")
print(missing_values)


Missing values in each column:
Unnamed: 0           0
Name                 0
Location             0
Year                 0
Kilometers_Driven    0
Fuel_Type            0
Transmission         0
Owner_Type           0
Mileage              0
Engine               0
Power                0
Seats                0
Price                0
dtype: int64


b) Remove the units from some of the attributes and only keep the numerical values (for
example remove kmpl from “Mileage”, CC from “Engine”, bhp from “Power”, and lakh from
“New_price”). (4 points)

In [7]:
df['Mileage'] = df['Mileage'].astype(str).str.replace(' kmpl', '').str.replace(' km/kg', '')
df['Engine'] = df['Engine'].astype(str).str.replace(' CC', '')
df['Power'] = df['Power'].astype(str).str.replace(' bhp', '')

In [8]:
df['Mileage'] = pd.to_numeric(df['Mileage'], errors='coerce')
df['Engine'] = pd.to_numeric(df['Engine'], errors='coerce')
df['Power'] = pd.to_numeric(df['Power'], errors='coerce')

In [9]:
df.head(5)

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67,1582.0,126.20,5.0,12.50
1,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13.00,1199.0,88.70,5.0,4.50
2,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77,1248.0,88.76,7.0,6.00
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.20,1968.0,140.80,5.0,17.74
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08,1461.0,63.10,5.0,3.50


C) Change the categorical variables (“Fuel_Type” and “Transmission”) into numerical one hot
encoded value. (4 points).

In [10]:
# Map the values: 1 for Manual, 0 for Automatic
df['Transmission_(1:Manual & 0:Automatic)'] = df['Transmission'].map({'Manual': 1, 'Automatic': 0})

# Map fuel types
fuel_map = {'Petrol': 0,
            'Diesel': 1,
            'Electric': 2}
df['Fuel_Type_(0:Petrol,1:Diesel & 2:Electric)'] = df['Fuel_Type'].map(fuel_map)

# Map owner types
owner_map = {
    'First': 1,
    'Second': 2,
    'Third': 3,
    'Fourth & Above': 4
}
df['Owner_Type_(1:First_2:Second_3:Third_4/4+:Fourth & Above)'] = df['Owner_Type'].map(owner_map)

# Droping the old columns
df.drop(columns=['Transmission'], inplace=True)
df.drop(columns=['Fuel_Type'], inplace=True)
df.drop(columns=['Owner_Type'], inplace=True)
df.head(5)

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Mileage,Engine,Power,Seats,Price,Transmission_(1:Manual & 0:Automatic),"Fuel_Type_(0:Petrol,1:Diesel & 2:Electric)",Owner_Type_(1:First_2:Second_3:Third_4/4+:Fourth & Above)
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,19.67,1582.0,126.20,5.0,12.50,1,1,1
1,2,Honda Jazz V,Chennai,2011,46000,13.00,1199.0,88.70,5.0,4.50,1,0,1
2,3,Maruti Ertiga VDI,Chennai,2012,87000,20.77,1248.0,88.76,7.0,6.00,1,1,1
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,15.20,1968.0,140.80,5.0,17.74,0,1,2
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,23.08,1461.0,63.10,5.0,3.50,1,1,1


d) Create one more feature and add this column to the dataset (you can use mutate function in
R for this). For example, you can calculate the current age of the car by subtracting “Year” value
from the current year. (4 points)

In [11]:
# Extract the first word from the 'Name' column as the company name
df['Company'] = df['Name'].str.split().str[0]
df.head(5)

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Mileage,Engine,Power,Seats,Price,Transmission_(1:Manual & 0:Automatic),"Fuel_Type_(0:Petrol,1:Diesel & 2:Electric)",Owner_Type_(1:First_2:Second_3:Third_4/4+:Fourth & Above),Company
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,19.67,1582.0,126.20,5.0,12.50,1,1,1,Hyundai
1,2,Honda Jazz V,Chennai,2011,46000,13.00,1199.0,88.70,5.0,4.50,1,0,1,Honda
2,3,Maruti Ertiga VDI,Chennai,2012,87000,20.77,1248.0,88.76,7.0,6.00,1,1,1,Maruti
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,15.20,1968.0,140.80,5.0,17.74,0,1,2,Audi
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,23.08,1461.0,63.10,5.0,3.50,1,1,1,Nissan


e) Perform select, filter, rename, mutate, arrange and summarize with group by operations (or
their equivalent operations in python) on this dataset. (4 points)
Submission:

In [12]:
# Select a subset of relevant columns
print(df.columns.tolist())

df_selected = df[['Company', 'Year', 'Mileage', 'Price']]


['Unnamed: 0', 'Name', 'Location', 'Year', 'Kilometers_Driven', 'Mileage', 'Engine', 'Power', 'Seats', 'Price', 'Transmission_(1:Manual & 0:Automatic)', 'Fuel_Type_(0:Petrol,1:Diesel & 2:Electric)', 'Owner_Type_(1:First_2:Second_3:Third_4/4+:Fourth & Above)', 'Company']


In [15]:
# Filter cars with Mileage > 18 and Fuel_Type is Petrol
df_filtered = df[(df['Mileage'] > 18) & (df['Fuel_Type_(0:Petrol,1:Diesel & 2:Electric)'] == 0)]
print("Filtered Data:\n", df_filtered.head())

Filtered Data:
     Unnamed: 0                                     Name   Location  Year  \
8           10                         Maruti Ciaz Zeta      Kochi  2018   
21          23                    Hyundai i20 1.2 Magna    Kolkata  2010   
25          27            Maruti Alto K10 2010-2014 VXI  Hyderabad  2013   
35          37  Mahindra KUV 100 mFALCON G80 K6 5str AW      Delhi  2017   
38          40                          Maruti Alto LXi      Delhi  2008   

    Kilometers_Driven  Mileage  Engine   Power  Seats  Price  \
8               25692    21.56  1462.0  103.25    5.0   9.95   
21              45807    18.50  1197.0   80.00    5.0   1.87   
25              54000    20.92   998.0   67.10    5.0   2.75   
35              52000    18.15  1198.0   82.00    5.0   4.85   
38              90000    19.70   796.0   46.30    5.0   1.25   

    Transmission_(1:Manual & 0:Automatic)  \
8                                       1   
21                                      1   
25     

In [16]:
# Rename columns for clarity
df_renamed = df.rename(columns={
    'Fuel_Type': 'FuelType',
    'Transmission (1:Manual & 0:Automatic)': 'GearType (1:Manual & 0:Automatic)',
    'Price': 'SellingPrice'
})
print("Renamed Data:\n", df_renamed.head())

Renamed Data:
    Unnamed: 0                              Name    Location  Year  \
0           1  Hyundai Creta 1.6 CRDi SX Option        Pune  2015   
1           2                      Honda Jazz V     Chennai  2011   
2           3                 Maruti Ertiga VDI     Chennai  2012   
3           4   Audi A4 New 2.0 TDI Multitronic  Coimbatore  2013   
4           6            Nissan Micra Diesel XV      Jaipur  2013   

   Kilometers_Driven  Mileage  Engine   Power  Seats  SellingPrice  \
0              41000    19.67  1582.0  126.20    5.0         12.50   
1              46000    13.00  1199.0   88.70    5.0          4.50   
2              87000    20.77  1248.0   88.76    7.0          6.00   
3              40670    15.20  1968.0  140.80    5.0         17.74   
4              86999    23.08  1461.0   63.10    5.0          3.50   

   Transmission_(1:Manual & 0:Automatic)  \
0                                      1   
1                                      1   
2                

In [17]:
# Add a new column: Age of car
current_year = 2025
df['Car_Age'] = current_year - df['Year']
print("Data with Age Column:\n", df.head())

Data with Age Column:
    Unnamed: 0                              Name    Location  Year  \
0           1  Hyundai Creta 1.6 CRDi SX Option        Pune  2015   
1           2                      Honda Jazz V     Chennai  2011   
2           3                 Maruti Ertiga VDI     Chennai  2012   
3           4   Audi A4 New 2.0 TDI Multitronic  Coimbatore  2013   
4           6            Nissan Micra Diesel XV      Jaipur  2013   

   Kilometers_Driven  Mileage  Engine   Power  Seats  Price  \
0              41000    19.67  1582.0  126.20    5.0  12.50   
1              46000    13.00  1199.0   88.70    5.0   4.50   
2              87000    20.77  1248.0   88.76    7.0   6.00   
3              40670    15.20  1968.0  140.80    5.0  17.74   
4              86999    23.08  1461.0   63.10    5.0   3.50   

   Transmission_(1:Manual & 0:Automatic)  \
0                                      1   
1                                      1   
2                                      1   
3      

In [18]:
# Arranging cars by Price descending
df_sorted = df.sort_values(by='Price', ascending=False)
print("Sorted Data:\n", df_sorted.head())

Sorted Data:
       Unnamed: 0                                         Name    Location  \
3952        4079  Land Rover Range Rover 3.0 Diesel LWB Vogue   Hyderabad   
5620        5781                   Lamborghini Gallardo Coupe       Delhi   
5752        5919                       Jaguar F Type 5.0 V8 S   Hyderabad   
1457        1505              Land Rover Range Rover Sport SE       Kochi   
1917        1974                           BMW 7 Series 740Li  Coimbatore   

      Year  Kilometers_Driven  Mileage  Engine  Power  Seats   Price  \
3952  2017              25000    13.33  2993.0  255.0    5.0  160.00   
5620  2011               6500     6.40  5204.0  560.0    2.0  120.00   
5752  2015               8000    12.50  5000.0  488.1    2.0  100.00   
1457  2019              26013    12.65  2993.0  255.0    5.0   97.07   
1917  2018              28060    12.05  2979.0  320.0    5.0   93.67   

      Transmission_(1:Manual & 0:Automatic)  \
3952                                      0

In [19]:
# Group by Fuel Type and summarize average price and average mileage
df_summary = df.groupby('Fuel_Type_(0:Petrol,1:Diesel & 2:Electric)').agg({
    'Price': 'mean',
    'Mileage': 'mean',
    'Power': 'mean'
}).reset_index().rename(columns={
    'Price': 'Avg_Price',
    'Mileage': 'Avg_Mileage',
    'Power': 'Avg_Power'
})
print(df_summary)

   Fuel_Type_(0:Petrol,1:Diesel & 2:Electric)  Avg_Price  Avg_Mileage  \
0                                           0   5.756688    17.576509   
1                                           1  12.960686    18.652661   

    Avg_Power  
0   97.281872  
1  127.867492  
